In [ ]:
# /kaggle/input/aiku-pokemon/pokemon_metadata_full.json
# /kaggle/input/aiku-pokemon/pokemonimage
# /kaggle/input/aiku-pokemon/pokemonimage/0001.png

'''
# 한국어 sbert
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
model = AutoModel.from_pretrained("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

# 한국어 klip
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification

processor = AutoProcessor.from_pretrained("Bingsu/clip-vit-large-patch14-ko")
model = AutoModelForZeroShotImageClassification.from_pretrained("Bingsu/clip-vit-large-patch14-ko")
'''

# 텍스트 임베딩 (KR-SBERT)

In [ ]:
!pip install sentence-transformers --quiet

import json
from sentence_transformers import SentenceTransformer
import numpy as np

# 1. 메타데이터 로드
metadata_path = '/kaggle/input/aiku-pokemon/pokemon_metadata_full.json'
with open(metadata_path, 'r', encoding='utf-8') as f:
    metadata = json.load(f)

# 2. 통합 설명문 준비
def make_full_description(entry):
    desc = entry.get('full_description_ko', entry.get('description_ko', ''))
    if desc:
        return desc
    types = ', '.join(entry.get('types', []))
    classi = entry.get('classification', '')
    return f"{entry.get('description_ko', '')} 타입: {types}. 분류: {classi}."

for entry in metadata:
    entry['full_description_ko'] = make_full_description(entry)

# 3. KR-SBERT 모델로 임베딩 생성
sbert_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
text_embeddings = {}

for entry in metadata:
    text = entry['full_description_ko']
    emb = sbert_model.encode(text)
    text_embeddings[entry['id']] = emb

np.save('/kaggle/working/krsbert_text_embeddings.npy', text_embeddings)


# 이미지 임베딩 (한국어 CLIP)

In [ ]:
!pip install transformers timm --quiet

from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch
import numpy as np
import os

model = CLIPModel.from_pretrained("Bingsu/clip-vit-large-patch14-ko")
processor = CLIPProcessor.from_pretrained("Bingsu/clip-vit-large-patch14-ko")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

image_folder = '/kaggle/input/aiku-pokemon/pokemonimage'
image_embeddings = {}

for entry in metadata:
    img_path = os.path.join(image_folder, f"{entry['id']}.png")
    if os.path.exists(img_path):
        image = Image.open(img_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            emb = model.get_image_features(**inputs)
            emb = emb.cpu().numpy().flatten()
            image_embeddings[entry['id']] = emb
    else:
        print(f"누락: {img_path}")

np.save('/kaggle/working/clip_image_embeddings.npy', image_embeddings)


# FAISS

In [ ]:
!pip install faiss-cpu --quiet


In [ ]:
import numpy as np

# SBERT 임베딩
sbert_text_embeddings = np.load('/kaggle/input/pokemon-embedding-1/krsbert_text_embeddings.npy', allow_pickle=True).item()
# CLIP 임베딩
clip_img_embeddings = np.load('/kaggle/input/pokemon-embedding-1/clip_image_embeddings.npy', allow_pickle=True).item()

# id 리스트 만들기 (항상 id 순서 통일!)
id_list = list(sbert_text_embeddings.keys())


In [ ]:
import json

with open('/kaggle/input/aiku-pokemon/pokemon_metadata_full.json', 'r', encoding='utf-8') as f:
    metadata = json.load(f)

# id→메타데이터 dict 만들기
id2meta = {entry['id']: entry for entry in metadata}


In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import CLIPProcessor, CLIPModel
import torch

# SBERT(한국어 특화)
sbert_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# CLIP(한국어)
clip_model = CLIPModel.from_pretrained("Bingsu/clip-vit-large-patch14-ko")
clip_processor = CLIPProcessor.from_pretrained("Bingsu/clip-vit-large-patch14-ko")
clip_model = clip_model.to('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
image_folder = '/kaggle/input/aiku-pokemon/pokemonimage'


In [ ]:
import faiss
import numpy as np

# id_list, sbert_text_embeddings, clip_img_embeddings는 이미 불러온 상태라고 가정

# 1. SBERT 임베딩 배열화 (N, D)
sbert_emb_matrix = np.stack([sbert_text_embeddings[pid] for pid in id_list])
sbert_emb_matrix = sbert_emb_matrix / np.linalg.norm(sbert_emb_matrix, axis=1, keepdims=True)  # 정규화

# 2. CLIP 임베딩 배열화 (N, D)
clip_emb_matrix = np.stack([clip_img_embeddings[pid] for pid in id_list])
clip_emb_matrix = clip_emb_matrix / np.linalg.norm(clip_emb_matrix, axis=1, keepdims=True)  # 정규화

# 3. FAISS 인덱스 각각 생성 (Inner Product == Cosine 유사도)
sbert_index = faiss.IndexFlatIP(sbert_emb_matrix.shape[1])
clip_index  = faiss.IndexFlatIP(clip_emb_matrix.shape[1])

sbert_index.add(sbert_emb_matrix.astype(np.float32))
clip_index.add(clip_emb_matrix.astype(np.float32))
